In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.preprocessing import StandardScaler
from t2e_utils import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
palette=sns.color_palette("RdBu_r", 50)
import pickle
import random
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import gc
from math import ceil
import tensorflow as tf
import wtte.wtte as wtte
from keras.models import Sequential, load_model,Model
from keras.layers import Dense,LSTM,GRU,Activation,Masking,BatchNormalization,Lambda,Input
from keras import backend as K
from keras import callbacks
from keras.optimizers import RMSprop,adam,Nadam
from keras.callbacks import History, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
plt.style.use('seaborn-muted')
# # np.random.seed(2)
# # pd.set_option("display.max_rows",1000)
%matplotlib inline
%load_ext autoreload
%autoreload 2
import h5py

Using TensorFlow backend.


In [2]:
path = '../Tax_paper/code/output_files/results/'
a = os.path.join(path,'suffix_and_remaining_time_helpdesk.csv')
b = os.path.join(path,'suffix_and_remaining_time_bpi_12_w.csv')
c = os.path.join(path,'suffix_and_remaining_time_bpi_12_w_no_repeat.csv')
d = os.path.join(path,'suffix_and_remaining_time_env_permit.csv')

a = pd.read_csv(a)
b = pd.read_csv(b)
c = pd.read_csv(c)
d = pd.read_csv(d)

df_dict = {
    'a': a,
    'b': b,
    'c': c,
    'd': d
}
range_dict = {
    'a': range(2,8,1),
    'b': range(2,22,2),
    'c': range(2,12,2),
    'd': range(2,22,2)
}

In [ ]:
def time_features(case):
    endtime = case["CompleteTimestamp"].reset_index(drop=True)[len(case)-1]
    starttime = case["CompleteTimestamp"].reset_index(drop=True)[0]
    case["fvt1"] = case["CompleteTimestamp"].diff(periods=1)
    case["fvt2"] = case["CompleteTimestamp"].dt.hour
    case["fvt3"] = case["CompleteTimestamp"].dt.weekday*24 + case["CompleteTimestamp"].dt.hour
    case["T2E"] =  endtime - case["CompleteTimestamp"]
    return case

def xy_split(processed_dataset, resolution = 'hourly',n_steps=2):
    X = processed_dataset.groupby(["CaseID"]).apply(lambda df:extract_X(df, n_steps))
    X = np.concatenate(X.values)
    y = processed_dataset.groupby(["CaseID"]).apply(lambda df:extract_y(df, n_steps, resolution = resolution))
    y = np.concatenate(y.values)
    y = y.reshape(y.shape[0],1)
    y = np.append(y,np.ones_like(y),axis=1)
    return X,y

def extract_X(df,n_steps):
    feature_idx = np.concatenate(\
               np.where(df.columns.str.contains('ActivityID_')) + \
#                np.where(df_helpdesk_preprocessed.columns.str.contains('weekday_')) + \
               np.where(df.columns.str.contains('fvt'))
              )
    x = []
    steps = len(df)-n_steps+1
    steps = 1
    for i in range(steps):
        x.append(df.values[i:i+n_steps,feature_idx])
    return np.array(x)

def extract_y(df,n_steps,resolution='hourly'):
    y = []
    if resolution == 'hourly':
        time_idx = np.where(df.columns.str.contains("H2E"))[0]
    else:
        time_idx = np.where(df.columns.str.contains("D2E"))[0]
    steps = len(df)-n_steps+1
    steps = 1
    for i in range(steps):
        y.append(df.values[i+n_steps-1,time_idx])
    return np.array(y)

def preprocess(dataset, min_case_length = 3, name= "Helpdesk Dataset"):
    tmp = dataset.groupby(["CaseID"]).count()
    to_drop = list(tmp.loc[tmp["ActivityID"] < min_case_length].index)
    dataset.CompleteTimestamp = pd.to_datetime(dataset.CompleteTimestamp)
    dataset.sort_values(["CaseID", "CompleteTimestamp"],ascending=True)
    dataset = dataset.loc[~dataset["CaseID"].isin(to_drop)].reset_index(drop=True)
    dataset = dataset.groupby("CaseID").apply(lambda case:time_features(case))
    dataset["D2E"] = dataset["T2E"].apply(lambda x:x.days)
    dataset["H2E"] = dataset["T2E"].apply(lambda x:x.total_seconds()/3600)
    dataset["fvt1"] = dataset["fvt1"].apply(lambda x:x.total_seconds()/3600)
    dataset.fvt1.fillna(0,inplace=True)
#     dataset["weekday"] = dataset["CompleteTimestamp"].dt.weekday
#     dummy1 = pd.get_dummies(dataset["weekday"],prefix="weekday",drop_first=True)
    dummy2 = pd.get_dummies(dataset["ActivityID"],prefix="ActivityID",drop_first=True)
#     dataset = pd.concat([dataset,dummy1,dummy2],axis=1)
    dataset = pd.concat([dataset,dummy2],axis=1)
    last_step = dataset.drop_duplicates(subset=["CaseID"],keep='last')["ActivityID"].index
    dataset = dataset.drop(last_step,axis=0).reset_index(drop=True)
    sc = StandardScaler()
    dataset["fvt1"] = sc.fit_transform(dataset["fvt1"].values.reshape(-1,1))
    dataset["fvt2"] = sc.fit_transform(dataset["fvt2"].values.reshape(-1,1))
    dataset["fvt3"] = sc.fit_transform(dataset["fvt3"].values.reshape(-1,1))
    return dataset